In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Depression"
cohort = "GSE81761"

# Input paths
in_trait_dir = "../../input/GEO/Depression"
in_cohort_dir = "../../input/GEO/Depression/GSE81761"

# Output paths
out_data_file = "../../output/preprocess/Depression/GSE81761.csv"
out_gene_data_file = "../../output/preprocess/Depression/gene_data/GSE81761.csv"
out_clinical_data_file = "../../output/preprocess/Depression/clinical_data/GSE81761.csv"
json_path = "../../output/preprocess/Depression/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene Expression Pathways Implicated in Posttraumatic Stress Disorder and Symptomatic Improvement"
!Series_summary	"Military service members often return from deployment with a multiplicity of injuries, including mild traumatic brain injury, depression, and sleep disorders, which obsures diagnosis of PTSD symptoms and complicates treatment of PTSD. In order to understand the biological mechanisms underlying PTSD, gene expression profiles of military service members with and without PTSD were compared. Additionally, gene expression was examined based on intrusion symptoms, a distinct subtype of PTSD symptoms, and on improvement of PTSD symptoms at a three month follow up. RNA was extracted from blood samples and hybridized to the HG-U133_Plus_2 Affymetrix chip."
!Series_overall_design	"Gene expression of subjects with PTSD (n=39) were compared to controls without PTSD (n=27) at baseline. Further analysis of gene expression for subjects with PTSD at 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, we can see this dataset contains gene expression data
# "RNA was extracted from blood samples and hybridized to the HG-U133_Plus_2 Affymetrix chip"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Depression/PTSD):
# Looking at the sample characteristics, we see "case/control: PTSD" and "case/control: No PTSD" in row 1
trait_row = 1

# For age:
# Age data is available in row 5 with multiple values
age_row = 5

# For gender:
# Gender data is available in row 4 as "Sex: Male" and "Sex: Female"
gender_row = 4

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait data to binary: 1 for PTSD, 0 for No PTSD."""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
        if "PTSD" in value and "No PTSD" not in value:
            return 1
        elif "No PTSD" in value:
            return 0
    return None

def convert_age(value):
    """Convert age data to continuous values."""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
        try:
            return float(value)
        except (ValueError, TypeError):
            return None
    return None

def convert_gender(value):
    """Convert gender data to binary: 0 for female, 1 for male."""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
        if "female" in value:
            return 0
        elif "male" in value:
            return 1
    return None

# 3. Save Metadata
# Determine if trait data is available for initial filtering
is_trait_available = trait_row is not None
# Save metadata for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Check what files are available in the input directory
    available_files = os.listdir(in_cohort_dir)
    print(f"Available files in {in_cohort_dir}: {available_files}")
    
    # Look for the clinical data file with various possible names
    clinical_file = None
    potential_names = ["clinical_data.csv", "sample_characteristics.csv", "characteristics.csv", 
                       "phenotype.csv", "clinical.csv"]
    for filename in available_files:
        if filename in potential_names or "clinical" in filename.lower() or "sample" in filename.lower():
            clinical_file = os.path.join(in_cohort_dir, filename)
            print(f"Found clinical data file: {clinical_file}")
            break
    
    if clinical_file and os.path.exists(clinical_file):
        # Use the library function to extract clinical features
        clinical_data = pd.read_csv(clinical_file)
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,  # Using the variable from context even though data is PTSD
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the output
        print("Preview of selected clinical features:")
        print(preview_df(selected_clinical_df))
        
        # Save clinical data to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    else:
        print(f"No clinical data file found in {in_cohort_dir}. Available files: {available_files}")


Available files in ../../input/GEO/Depression/GSE81761: ['GSE81761_family.soft.gz', 'GSE81761_series_matrix.txt.gz']
No clinical data file found in ../../input/GEO/Depression/GSE81761. Available files: ['GSE81761_family.soft.gz', 'GSE81761_series_matrix.txt.gz']


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Depression/GSE81761/GSE81761_series_matrix.txt.gz


Gene data shape: (54675, 109)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs, not human gene symbols
# The format "XXXXXX_at" is characteristic of Affymetrix microarray probe identifiers
# We will need to map these to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# Check if there are any platforms defined in the SOFT file that might contain annotation data
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Look for platform sections in the SOFT file
platform_sections = re.findall(r'^!Platform_title\s*=\s*(.+)$', soft_content, re.MULTILINE)
if platform_sections:
    print(f"Platform title found: {platform_sections[0]}")

# Try to extract more annotation data by reading directly from the SOFT file
# Look for lines that might contain gene symbol mappings
symbol_pattern = re.compile(r'ID_REF\s+Symbol|ID\s+Gene Symbol', re.IGNORECASE)
annotation_lines = []
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if symbol_pattern.search(line):
            annotation_lines.append(line)
            # Collect the next few lines to see the annotation structure
            for _ in range(10):
                annotation_lines.append(next(f, ''))

if annotation_lines:
    print("Found potential gene symbol mappings:")
    for line in annotation_lines:
        print(line.strip())

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("\nGene annotation preview:")
print(preview_df(gene_annotation, n=10))

# If we need an alternative source of mapping, check if there are any other annotation files in the cohort directory
cohort_files = os.listdir(in_cohort_dir)
annotation_files = [f for f in cohort_files if 'annotation' in f.lower() or 'platform' in f.lower()]
if annotation_files:
    print("\nAdditional annotation files found in the cohort directory:")
    for file in annotation_files:
        print(file)


Platform title found: [HG-U133_Plus_2] Affymetrix Human Genome U133 Plus 2.0 Array



Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861', 'L13852', 'X55005', 'X79510', 'M21121', 'J02843'], 'SPOT_ID': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprieta

### Step 6: Gene Identifier Mapping

In [7]:
# Looking at the gene annotation dataframe, we can see:
# - 'ID' column appears to match the gene identifiers shown in gene_data.index
# - 'Gene Symbol' column contains the gene symbols we need to map to

# Extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
print(f"Gene mapping shape: {gene_mapping.shape}")
print("First few rows of gene mapping dataframe:")
print(gene_mapping.head())

# Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols in the mapped gene expression data:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping shape: (45782, 2)
First few rows of gene mapping dataframe:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Gene expression data shape after mapping: (21278, 109)
First few gene symbols in the mapped gene expression data:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Depression/gene_data/GSE81761.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data - we already did this in step 6
# No need to do it again
print(f"Gene data already normalized and saved to {out_gene_data_file}")

# 2. Need to recreate clinical data since it wasn't properly saved in step 2
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Extract clinical features
def convert_trait(value):
    """Convert trait data to binary: 1 for PTSD, 0 for No PTSD."""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
        if "PTSD" in value and "No PTSD" not in value:
            return 1
        elif "No PTSD" in value:
            return 0
    return None

# Use row 1 as identified in step 2 for the trait (case/control PTSD status)
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,  # Using the trait variable from context
    trait_row=1,  # Using row 1 for PTSD status as identified in step 2
    convert_trait=convert_trait,
    age_row=5,    # Age data is in row 5
    convert_age=lambda value: float(value.split(":", 1)[1].strip()) if isinstance(value, str) and ":" in value else None,
    gender_row=4, # Gender data is in row 4
    convert_gender=lambda value: 0 if isinstance(value, str) and ":" in value and "female" in value.split(":", 1)[1].lower() else (
                               1 if isinstance(value, str) and ":" in value and "male" in value.split(":", 1)[1].lower() else None)
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save clinical data for future reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from peripheral blood of military service members with and without PTSD, though labeled for Depression in our analysis pipeline."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Gene data already normalized and saved to ../../output/preprocess/Depression/gene_data/GSE81761.csv


Selected clinical data shape: (3, 109)
Clinical data preview:
{'GSM2175165': [1.0, 30.0, 1.0], 'GSM2175166': [0.0, 38.0, 1.0], 'GSM2175167': [1.0, 39.0, 1.0], 'GSM2175168': [1.0, 38.0, 1.0], 'GSM2175169': [1.0, 23.0, 1.0], 'GSM2175170': [0.0, 48.0, 1.0], 'GSM2175171': [0.0, 49.0, 1.0], 'GSM2175172': [0.0, 34.0, 1.0], 'GSM2175173': [1.0, 33.0, 1.0], 'GSM2175174': [1.0, 45.0, 1.0], 'GSM2175175': [1.0, 25.0, 1.0], 'GSM2175176': [0.0, 25.0, 1.0], 'GSM2175177': [1.0, 30.0, 1.0], 'GSM2175178': [1.0, 39.0, 1.0], 'GSM2175179': [1.0, 23.0, 1.0], 'GSM2175180': [1.0, 22.0, 1.0], 'GSM2175181': [1.0, 46.0, 1.0], 'GSM2175182': [0.0, 35.0, 1.0], 'GSM2175183': [0.0, 22.0, 1.0], 'GSM2175184': [1.0, 23.0, 1.0], 'GSM2175185': [0.0, 48.0, 1.0], 'GSM2175186': [1.0, 23.0, 1.0], 'GSM2175187': [0.0, 49.0, 1.0], 'GSM2175188': [1.0, 38.0, 1.0], 'GSM2175189': [1.0, 25.0, 1.0], 'GSM2175190': [1.0, 33.0, 1.0], 'GSM2175191': [1.0, 30.0, 1.0], 'GSM2175192': [0.0, 36.0, 1.0], 'GSM2175193': [0.0, 43.0, 1.0], 'GSM21751

Data shape after handling missing values: (109, 21281)
For the feature 'Depression', the least common label is '0.0' with 47 occurrences. This represents 43.12% of the dataset.
The distribution of the feature 'Depression' in this dataset is fine.

Quartiles for 'Age':
  25%: 26.0
  50% (Median): 33.0
  75%: 39.0
Min: 21.0
Max: 52.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 5 occurrences. This represents 4.59% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Depression/GSE81761.csv
